Парсер prodoctorov.ru. Часть 1, собирает ссылки.

Последнее обновление: 2021_12_28

## Библиотеки и настройки ##

In [ ]:
#Загружаю нужные библиотеки

import pandas as pd
import math

# регулярки
import re

# для юзер-агента
from fake_useragent import UserAgent

# прогресс-бар
from tqdm.notebook import trange, tqdm

# для пауз при парсинге
from time import sleep

# селениум
from selenium import webdriver

from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# настройки юзер-агента

ua = UserAgent()

# юзер-агент для Хрома
user_agent = ua.chrome

In [ ]:
test_url = 'https://intoli.com/blog/not-possible-to-block-chrome-headless/chrome-headless-test.html'

In [ ]:
# путь к файлам

file_path = 'C:/00_Data/stom/'
proxy_list_path = 'C:/work/proxy_list.txt' # путь и названия со списком прокси

In [ ]:
# загружаю список прокси
# список простой текстовый файл, который содержит прокси в формате: адрес:порт логин:пароль

proxy_list = []

with open(proxy_list_path, 'r') as f:
    for line in f:
        proxy_list += [line.split()]
        
print('Рабочих прокси — ' + str(len(proxy_list)))

In [ ]:
# номер прокси

print('Номер прокси, от 0 до ' + str(len(proxy_list) - 1))
proxy_number = int(input())

In [ ]:
# настройки веб-драйвера, чтобы скрыть автоматическое управление

opt = webdriver.ChromeOptions()
opt.add_argument(f'user-agent={user_agent}')
opt.add_argument('--disable-blink-features')
opt.add_argument('--disable-blink-features=AutomationControlled')
opt.add_argument(f'--proxy-server={proxy_list[proxy_number][0]}') # "девственный" прокси(:
opt.add_experimental_option("excludeSwitches", ["enable-automation"]) # отключает строку про удаленное управление
opt.add_experimental_option('useAutomationExtension', False)

In [ ]:
print('Login: ' + proxy_list[proxy_number][1])
print('Pass: ' + proxy_list[proxy_number][2])

In [ ]:
# запускаю драйвер, проверяю настройки

driver = webdriver.Chrome(options=opt)
driver.get(test_url)
driver.maximize_window() # раскрываю окно на максимум

## Собираю ссылки ##

In [ ]:
# нужный город латиницей, транслитом как на сайте

need_city = input()

In [ ]:
# ссылка, у Продокторов очень простая навигация (:

url_part_1 = f'https://prodoctorov.ru/{need_city}/top/chastnaya-stomatologiya/?page='

In [ ]:
# по умолчанию Москва

driver.get(url_part_1 + '1')

Считаю количество страниц.

На странице 20 клиник, на первую страницу Prodoctorov добавляет 3 платных ссылки (они дублируются).

In [ ]:
# Считаю количество страниц

clinic_number = int(driver
                    .find_element_by_class_name('p-page__title')
                    .get_attribute('data-counter')
                    .replace(')', '').replace('(', '').replace(' ', ''))
page_number = (clinic_number + 3) // 20 + 1

print('Количество клиник: ' + str(clinic_number))
print('Количество страниц: ' + str(page_number))

In [ ]:
data = [] # список для данных

for i in tqdm(range(1, page_number + 1)): # цикл для сбора ссылок
    
    driver.get(url_part_1 + str(i))
    sleep(2)
    
    links = driver.find_elements_by_class_name('b-card__name') # беру блоки со страницы
    
    for link in links: # вытаскиваю ссылки
        clinic = link.text
        clinic_page = link.find_element_by_tag_name('a').get_attribute('href')
        
        data.append([clinic, clinic_page]) # добавляю данные в список

In [ ]:
# Записываю в датафрейм

head_list = ['clinic', 'clinic_page']
work_links = pd.DataFrame(data, columns=head_list)

In [ ]:
work_links.shape

In [ ]:
# удаляю дубликаты

work_links.drop_duplicates(inplace=True)

In [ ]:
work_links.shape

In [ ]:
work_links.to_excel(file_path + f'prodoctorov_urls_{need_city}.xlsx', encoding='utf8', index=False)

In [ ]:
driver.quit()